<a href="https://colab.research.google.com/github/RedBatProject/Works/blob/main/work81image_done.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q kaggle
!mkdir ~/.kaggle
! cp /content/drive/MyDrive/kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!kaggle competitions download -c cassava-disease

100% 2.30G/2.30G [02:25<00:00, 22.4MB/s]
100% 2.30G/2.30G [02:25<00:00, 17.0MB/s]


In [ ]:
# !mv /content/cassava-disease.zip /content/drive/MyDrive/work81image/

In [ ]:
!unzip /content/drive/MyDrive/work81image/cassava-disease.zip

Archive:  /content/drive/MyDrive/work81image/cassava-disease.zip
  inflating: extraimages.zip         
  inflating: random.txt              
  inflating: sample_submission_file.csv  
  inflating: test.zip                
  inflating: train.zip               


In [ ]:
# !unzip /content/train.
!unzip /content/test.zip

Archive:  /content/test.zip
   creating: test/
   creating: test/0/
  inflating: test/0/test-img-2397.jpg  
  inflating: test/0/test-img-3089.jpg  
  inflating: test/0/test-img-1846.jpg  
  inflating: test/0/test-img-863.jpg  
  inflating: test/0/test-img-877.jpg  
  inflating: test/0/test-img-1852.jpg  
  inflating: test/0/test-img-2383.jpg  
  inflating: test/0/test-img-687.jpg  
  inflating: test/0/test-img-693.jpg  
  inflating: test/0/test-img-1885.jpg  
  inflating: test/0/test-img-2354.jpg  
  inflating: test/0/test-img-2432.jpg  
  inflating: test/0/test-img-678.jpg  
  inflating: test/0/test-img-2426.jpg  
  inflating: test/0/test-img-3738.jpg  
  inflating: test/0/test-img-2340.jpg  
  inflating: test/0/test-img-1649.jpg  
  inflating: test/0/test-img-1891.jpg  
  inflating: test/0/test-img-3076.jpg  
  inflating: test/0/test-img-2368.jpg  
  inflating: test/0/test-img-1661.jpg  
  inflating: test/0/test-img-122.jpg  
  inflating: test/0/test-img-644.jpg  
  inflating: test/0

In [ ]:
import os
import pandas as pd
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_path = '/content/train/'   # cmd, healthy, cgm, cbsd, cbb 2658+316+773+1443+466 == 6122
# test_path = '/content/test/0'
print(len(os.listdir(train_path)))
# print(len(os.listdir(test_path)))

5
3774


In [ ]:
split_size = 4897 #80-20
BATCH_SIZE = 70
train_dir = train_path
train_datagen = ImageDataGenerator(
    rescale = 1.0/255.0,
    rotation_range = 40,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    fill_mode = 'nearest',
    validation_split = 0.25
)

validation_datagen = ImageDataGenerator(
    rescale = 1.0/255.0,
    rotation_range = 40,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    fill_mode = 'nearest'
)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    batch_size = BATCH_SIZE,
    class_mode = 'sparse',
    target_size = (150,150),
    shuffle = True,
    subset = 'training'
)

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    batch_size = BATCH_SIZE,
    class_mode = 'sparse',
    target_size = (150,150),
    shuffle = True,
    subset = 'validation'
)

Found 4244 images belonging to 5 classes.
Found 1412 images belonging to 5 classes.


In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape = (150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(16, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])

model.compile(optimizer=RMSprop(lr=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 64)      1792      
                                                                 
 max_pooling2d (MaxPooling2  (None, 74, 74, 64)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 32)        18464     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 36, 36, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 34, 34, 16)        4624      
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 17, 17, 16)        0

In [ ]:
history = model.fit_generator(train_generator,
                             epochs=30,
                             verbose=1,
                             validation_data=validation_generator) #vadidation_generator

<ipython-input-13-6d79790d1226>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_generator,


Epoch 1/30
61/61 [==============================] - 66s 989ms/step - loss: 1.3977 - accuracy: 0.4592 - val_loss: 1.3221 - val_accuracy: 0.4703
Epoch 2/30
61/61 [==============================] - 51s 843ms/step - loss: 1.3282 - accuracy: 0.4760 - val_loss: 1.2875 - val_accuracy: 0.4809
Epoch 3/30
61/61 [==============================] - 50s 820ms/step - loss: 1.2712 - accuracy: 0.5113 - val_loss: 1.1776 - val_accuracy: 0.5673
Epoch 4/30
61/61 [==============================] - 51s 848ms/step - loss: 1.2225 - accuracy: 0.5450 - val_loss: 1.1627 - val_accuracy: 0.5552
Epoch 5/30
61/61 [==============================] - 52s 854ms/step - loss: 1.1908 - accuracy: 0.5575 - val_loss: 1.1260 - val_accuracy: 0.5822
Epoch 6/30
61/61 [==============================] - 52s 849ms/step - loss: 1.1671 - accuracy: 0.5719 - val_loss: 1.1670 - val_accuracy: 0.5531
Epoch 7/30
61/61 [==============================] - 52s 853ms/step - loss: 1.1472 - accuracy: 0.5815 - val_loss: 1.0905 - val_accuracy: 0.6147

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training Accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.figure()


plt.plot(epochs, loss, 'r', label = 'Training Loss')
plt.plot(epochs, val_loss, 'b', label = 'Validation Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.figure()

NameError: name 'history' is not defined

In [ ]:
#get data
train_paths=[]
train_labels=[]
for root,dir,files in os.walk("/content/train"):
    for file in files:
        train_paths.append(os.path.join(root,file))
        train_labels.append(root.split("/")[-1])

# test_paths=[]
# for root,dir,files in os.walk("../input/test"):
#     for file in files:
#         test_paths.append(os.path.join(root,file))

print(train_paths[:5])
# print(test_paths[:5])

In [ ]:
#function to preprocess images and labels
def preprocess_image(image,image_shape=[192,192]):
  image = tf.image.decode_jpeg(image, channels=3)
  image = tf.image.resize(image, image_shape)
  image /= 255.0  # normalize to [0,1] range
  return image


def load_and_preprocess_image(file,image_shape):
    image = tf.io.read_file(file)
    return preprocess_image(image,image_shape)

labs2index={'cbb':0, 'cbsd':1, 'cgm':2, 'cmd':3, 'healthy':4}
index2labs={0:'cbb', 1:'cbsd', 2:'cgm', 3:'cmd', 4:'healthy'}
def process_labels(lab):
    print(lab)
    return tf.one_hot(lab,depth=len(labs2index))

In [ ]:
#create tensorflow data objects
train_ds = tf.data.Dataset.from_tensor_slices(train_paths)
# test_ds=tf.data.Dataset.from_tensor_slices(test_paths)
train_image_ds = train_ds.map(lambda x:load_and_preprocess_image(x,image_shape=[192,192]),num_parallel_calls=AUTOTUNE)
# test_image_ds=test_ds.map(lambda x:load_and_preprocess_image(x,image_shape=[192,192]),num_parallel_calls=AUTOTUNE)

train_label_ds=tf.data.Dataset.from_tensor_slices([labs2index[i] for i in train_labels]).map(process_labels,num_parallel_calls=AUTOTUNE)
image_label_ds=tf.data.Dataset.zip((train_image_ds,train_label_ds))

BATCH_SIZE = 32

# Setting a shuffle buffer size as large as the dataset ensures that the data is
# completely shuffled.
ds = image_label_ds.shuffle(buffer_size=len(train_paths))
ds = ds.repeat()
ds = ds.batch(BATCH_SIZE)
# `prefetch` lets the dataset fetch batches, in the background while the model is training.
ds = ds.prefetch(buffer_size=AUTOTUNE)

# ds_test=test_image_ds.batch(BATCH_SIZE)
print(ds)
# print(test_ds)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    batch_size = BATCH_SIZE,
    class_mode = 'sparse',
    target_size = (192, 192),
    shuffle = True,
    subset = 'training'
)

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    batch_size = BATCH_SIZE,
    class_mode = 'sparse',
    target_size = (192, 192),
    shuffle = True,
    subset = 'validation'
)

Found 4244 images belonging to 5 classes.
Found 1412 images belonging to 5 classes.


In [ ]:
#use pretrained model
mobile_net = tf.keras.applications.MobileNetV2(input_shape=(192, 192, 3), include_top=False)
#mobile_net.trainable=False

def change_range(image,label):
  return 2*image-1, label

# keras_ds = ds.map(change_range)
# keras_ds = ds

# keras_test=ds_test.map(lambda x:2*x-1)


#image_batch, label_batch = next(iter(keras_ds))

model = tf.keras.Sequential([
    mobile_net,
    tf.keras.layers.GlobalAveragePooling2D(),
    # tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(5,activation=tf.nn.softmax)])

model.compile(optimizer=RMSprop(lr=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_192 (Func  (None, 6, 6, 1280)        2257984   
 tional)                                                         
                                                                 
 global_average_pooling2d_3  (None, 1280)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_5 (Dense)             (None, 128)               163968    
                                                                 
 dense_6 (Dense)             (None, 5)                 645       
                                                                 
Total params: 2422597 (9.24 MB)
Trainable params: 2388485 (9.11 MB)
Non-trainable params: 34112 (133.25 KB)
_________________________________________________________________
None


In [ ]:
model.layers[0].trainable = False

In [ ]:
model.compile(optimizer=RMSprop(lr=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_192 (Func  (None, 6, 6, 1280)        2257984   
 tional)                                                         
                                                                 
 global_average_pooling2d_3  (None, 1280)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_5 (Dense)             (None, 128)               163968    
                                                                 
 dense_6 (Dense)             (None, 5)                 645       
                                                                 
Total params: 2422597 (9.24 MB)
Trainable params: 164613 (643.02 KB)
Non-trainable params: 2257984 (8.61 MB)
_________________________________________________________________
None


In [ ]:
history = model.fit_generator(train_generator,
                             epochs=30,
                             verbose=1,
                             validation_data=validation_generator) #vadidation_generator

<ipython-input-33-6d79790d1226>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_generator,


Epoch 1/30
61/61 [==============================] - 75s 1s/step - loss: 1.2717 - accuracy: 0.5450 - val_loss: 0.9951 - val_accuracy: 0.6013
Epoch 2/30
61/61 [==============================] - 70s 1s/step - loss: 0.9472 - accuracy: 0.6305 - val_loss: 0.8609 - val_accuracy: 0.6756
Epoch 3/30
61/61 [==============================] - 69s 1s/step - loss: 0.8821 - accuracy: 0.6673 - val_loss: 0.8268 - val_accuracy: 0.6848
Epoch 4/30
61/61 [==============================] - 69s 1s/step - loss: 0.8414 - accuracy: 0.6852 - val_loss: 0.8503 - val_accuracy: 0.6884
Epoch 5/30
61/61 [==============================] - 69s 1s/step - loss: 0.8206 - accuracy: 0.6876 - val_loss: 0.7895 - val_accuracy: 0.7118
Epoch 6/30
38/61 [=================>............] - ETA: 19s - loss: 0.7820 - accuracy: 0.7062

In [ ]:
#fit model for 50 epochs
model.fit(keras_ds, epochs=50, steps_per_epoch=176)

In [ ]:
#make predictions
test_predictions=model.predict(keras_test,steps=int(np.ceil(len(test_paths)/BATCH_SIZE)))
predictions=np.argmax(test_predictions,axis=1)

In [ ]:
#make submission
my_submission = pd.DataFrame({'Category':[index2labs[j] for j in predictions],'Id':[i.split("/").pop() for i in test_paths]})
# you could use any filename. We choose submission here
my_submission.to_csv('submission.csv', index=False)
print(my_submission.head())

In [ ]:
# https://www.sciencedirect.com/science/article/pii/S1110866520301110
# https://www.kaggle.com/c/cassava-disease/data